<a href="https://colab.research.google.com/github/merlinymy/mealPlanGenerator/blob/main/related_df_created.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, json

In [2]:
df = pd.read_json('foundationDownload.json')

In [3]:
with open('foundationDownload.json', 'r') as f:
  data = json.load(f)

In [4]:
foundation_foods = data.get('FoundationFoods', [])

In [5]:
first_foundation_food = foundation_foods[0]
first_foundation_food.keys()

dict_keys(['foodClass', 'description', 'foodNutrients', 'foodAttributes', 'nutrientConversionFactors', 'isHistoricalReference', 'ndbNumber', 'dataType', 'foodCategory', 'fdcId', 'foodPortions', 'publicationDate', 'inputFoods'])

In [ ]:
## key values to keep
# description field is a type of <class 'str'>
# foodNutrients field is a type of <class 'list'>
# ndbNumber field is a type of <class 'int'>
# foodCategory field is a type of <class 'dict'>
# fdcId field is a type of <class 'int'>
# inputFoods field is a type of <class 'list'>

<class 'dict_keys'>


In [23]:
extracted_data = []
for item in foundation_foods:
  extracted_item = {
      'description': item.get('description', ''),
      'foodNutrients': item.get('foodNutrients', []),
      'ndbNumber': item.get('ndbNumber', None),
      'foodCategory': item.get('foodCategory', {}),
      'fdcId': item.get('fdcId', None),
      'inputFoods': item.get('inputFoods', [])
  }
  extracted_data.append(extracted_item)

# Creating a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)


In [26]:
# Extracting 'description' from 'foodCategory' and adding it as a new column
extracted_df['foodCategory'] = extracted_df['foodCategory'].apply(lambda x: x.get('description', ''))

# Creating separate DataFrames for 'foodNutrients' and 'inputFoods'
food_nutrients_data = []
input_foods_data = []

for index, row in extracted_df.iterrows():
    fdc_id = row['fdcId']

    # Extracting 'foodNutrients' details
    for nutrient in row['foodNutrients']:
        nutrient['fdcId'] = fdc_id
        food_nutrients_data.append(nutrient)

    # Extracting 'inputFoods' details
    for input_food in row['inputFoods']:
        input_food['fdcId'] = fdc_id
        input_foods_data.append(input_food)

# Creating DataFrames
food_nutrients_df = pd.DataFrame(food_nutrients_data)
input_foods_df = pd.DataFrame(input_foods_data)




In [17]:
# Simplifying the food_nutrients_df by removing unnecessary columns and extracting only 'name' and 'unitName' from 'nutrient'

# Dropping unwanted columns
simplified_nutrients_df = food_nutrients_df.drop(['dataPoints', 'foodNutrientDerivation', 'max', 'min', 'footnote', 'median'], axis=1)


In [18]:
# Extracting 'name' and 'unitName' from 'nutrient' column
simplified_nutrients_df['nutrient_name'] = simplified_nutrients_df['nutrient'].apply(lambda x: x.get('name', ''))
simplified_nutrients_df['nutrient_unit'] = simplified_nutrients_df['nutrient'].apply(lambda x: x.get('unitName', ''))

# Dropping the original 'nutrient' column
simplified_nutrients_df = simplified_nutrients_df.drop(['nutrient'], axis=1)

simplified_nutrients_df.head()  # Displaying the first few rows of the simplified DataFrame


,type,id,amount,fdcId,nutrient_name,nutrient_unit
0,FoodNutrient,2219707,3.0,321358,"Cryptoxanthin, beta",µg
1,FoodNutrient,2219708,0.0,321358,Lycopene,µg
2,FoodNutrient,2219709,1.3,321358,"Tocopherol, delta",mg
3,FoodNutrient,2219710,0.0,321358,"Tocotrienol, gamma",mg
4,FoodNutrient,2219711,0.0,321358,"Tocotrienol, delta",mg


In [30]:
main_df = extracted_df.drop(['foodNutrients','ndbNumber','inputFoods'], axis=1)

In [31]:
main_df

,description,foodCategory,fdcId
0,"Hummus, commercial",Legumes and Legume Products,321358
1,"Tomatoes, grape, raw",Vegetables and Vegetable Products,321360
2,"Beans, snap, green, canned, regular pack, drai...",Vegetables and Vegetable Products,321611
3,"Frankfurter, beef, unheated",Sausages and Luncheon Meats,323121
4,"Nuts, almonds, dry roasted, with salt added",Nut and Seed Products,323294
...,...,...,...
260,"Cheese, pasteurized process cheese food or pro...",Dairy and Egg Products,2647439
261,"Cheese, provolone, sliced",Dairy and Egg Products,2647440
262,"Cheese, oaxaca, solid",Dairy and Egg Products,2647441
263,"Cheese, queso fresco, solid",Dairy and Egg Products,2647442


In [32]:
simplified_nutrients_df

,type,id,amount,fdcId,nutrient_name,nutrient_unit
0,FoodNutrient,2219707,3.0,321358,"Cryptoxanthin, beta",µg
1,FoodNutrient,2219708,0.0,321358,Lycopene,µg
2,FoodNutrient,2219709,1.3,321358,"Tocopherol, delta",mg
3,FoodNutrient,2219710,0.0,321358,"Tocotrienol, gamma",mg
4,FoodNutrient,2219711,0.0,321358,"Tocotrienol, delta",mg
...,...,...,...,...,...,...
12908,FoodNutrient,33296880,NaN,2647443,Vitamin A,mg
12909,FoodNutrient,33296881,0.0,2647443,Vitamin D (D2 + D3),µg
12910,FoodNutrient,33296882,351.0,2647443,Energy (Atwater General Factors),kcal
12911,FoodNutrient,33296883,352.0,2647443,Energy (Atwater Specific Factors),kcal
